### Load Data

In [1]:
from rdflib import Graph

g = Graph()
g.parse("Data/claimskg_v1.0.ttl", format="turtle")

6606074

### Save graph as PKL

In [5]:
import pickle

filename = 'Data/ClaimsKG.pkl'
pickle.dump(g, open(filename, 'wb'))

### Load PKL

In [6]:
g2 = pickle.load(open("Data/ClaimsKG.pkl", 'rb'))

### Execute Query

In [8]:
qres = g2.query(
    """PREFIX schema: <http://schema.org/>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

       SELECT DISTINCT ?keywords WHERE {
          ?claim a schema:CreativeWork.
          ?claim schema:keywords ?keywords
       } LIMIT 100""")

for row in qres:
    print("%s" % row)

ASP Article, barack obama
Health Care,Immigration,Public Health
Children
Energy,Environment
Corporations,Economy,Gays and Lesbians,State Finances,States
Economy,Workers
Bankruptcy,Debt,Federal Budget,Financial Regulation,Retirement
Crime,Criminal Justice,Economy,Sexuality
Income,Small Business,Taxes
People
Elections,Immigration,Redistricting
Federal Budget,Government Efficiency
Guns
ASP Article, Not Necessarily The News
Job Accomplishments,Stimulus
Candidate Biography,Legal Issues
fast food rumors, food, italy, mcdonald's
Economy,Gays and Lesbians,Sexuality,States
Economy,Jobs,Workers
Elections
Education,Poverty,State Budget,State government spending ,Taxes,Urban
Elections,Transparency
Crime
ASP Article
Debates,Elections
Candidate Biography,Pundits
emergency alert system, false flag, fcc, FEMA, hoboken train crash, viral videos
barack obama, national anthem
Crime,Jobs
Trade
cancer, health, reader suggestion, vaccination
Health Care
Candidate Biography,Corrections and Updates
ASP Articl